<a href="https://colab.research.google.com/github/bjpark-forest/2023-1-Intro_ML/blob/main/KNN_iris_without_sklearn_KNNclassifier_Euclidiean_k4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Create/explaining a simple classification code with a decision rule similar to KNN

### 사전 준비
- 데이터 수집 : sklearn.datasets.load_iris
- 데이터 분할 : sklearn.model_selection.train_test_split

In [ ]:
# 파이썬 pandas 라이브러리를 불러오기(import pandas)
import pandas as pd
# 사이킷런에 준비된 데이터셋에서 아이리스 데이터를 불러오기(load_iris)
from sklearn.datasets import load_iris 
# 불러온 아이리스 데이터를 메모리에 객체(instance)로 만들기
iris = load_iris() 

In [ ]:
# iris.data와 iris.feature_names를 활용하여 데이터프레임 자료구조에 값을 넣은 후 df라는 변수에 할당하기
df = pd.DataFrame(data=iris.data, columns=iris.feature_names)

# 데이터프레임에 'target'이라는 새로운 열을 만들고, 해당 열의 값에 iris.target값을 대입하기
df['target'] = iris.target

In [ ]:
# df 데이터프레임의 열 이름을 아래와 같이 바꾸기
df.columns=['sl','sw','pl','pw','label']

In [ ]:
# 준비된 데이터를 학습용과 검증용으로 나누기
# 150개의 데이터 샘플을 학습용 80%(120개)와 검증용 20%(30개)로 나눔.
# random_state: 모델 결과 재현을 위해 생성 시킨 의사 난수(pseudo random number)
from sklearn.model_selection import train_test_split
train,test=train_test_split(df,test_size=0.2, random_state=42)

In [ ]:
# 학습용 데이터를 다시 특성 데이터(x_train)와 라벨 데이터(y_train)로 나누기
x_train=train.drop('label', axis=1)
y_train=train['label']

In [ ]:
#  검증용 데이터를 특성 데이터(x_test)와 라벨 데이터(y_train)으로 나누기
x_test=test.drop('label', axis=1)
y_test=test['label']

In [ ]:
# 학습용 데이터와 검증용 데이터 갯수 확인
len(x_train), len(y_train), len(x_test), len(y_test)

(120, 120, 30, 30)

# 방법1 ) 사이킷런의  KNeighborsClassifier 분류기를 사용하는 경우

#### sklearn 공식 문서의 KNN 구현 방법 
https://scikit-learn.org/stable/modules/generated/sklearn.neighbors.KNeighborsClassifier.html

- p: int, default=2
Power parameter for the Minkowski metric. When p = 1, this is equivalent to using manhattan_distance (l1), and euclidean_distance (l2) for p = 2. For arbitrary p, minkowski_distance (l_p) is used.

- metric: str or callable, default=’minkowski’
Metric to use for distance computation. Default is “minkowski”, which results in the standard Euclidean distance when p = 2.

* 민코프스키 거리(minkowski Distance) 
- n차원 민코프스키 공간에서의 거리 
- X = $({x_1, x_2, x_3,..., x_n})$, Y= $({y_1, y_2, y_3, ..., y_n})$ 
- D(X, Y) = $({{\vert{x_{i1} - x_{j1}}\vert}^p + {\vert{x_{i2} - x_{j2}}\vert}^p + ... + {\vert{x_{ip} - x_{jp}}\vert}^p })^{1\over p}$
- D(X, Y) = $(\displaystyle\sum_{i=1}^{n}{{\vert{x_{i} - y_{i}}\vert}^p  })^{1\over p}$

In [ ]:

from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier(n_neighbors=4)
knn.fit(x_train,y_train.values)
predictions = knn.predict(x_test)
print(predictions)
print(y_test.values)

[1 0 2 1 1 0 1 2 1 1 2 0 0 0 0 1 2 1 1 2 0 2 0 2 2 2 2 2 0 0]
[1 0 2 1 1 0 1 2 1 1 2 0 0 0 0 1 2 1 1 2 0 2 0 2 2 2 2 2 0 0]


In [ ]:
# 정확도: 일치하는 값의 갯수 / 전체 개수
accuracy = ((predictions == y_test.values).sum()) / len(predictions)
accuracy

1.0

# 방법2) 사이킷런을 사용하지 않고 직접 구현하는 경우

### Create/explaining a simple classification code with a decision rule similar to KNN

- 차원이 k인 두 샘플의 유클리드 거리(p=2)를 기반으로 모든 데이터 샘플간의 거리를 구함.
- D(X, Y) = $\sqrt {{\vert{x_1 - y_1}\vert}^2 + {\vert{x_2 - y_j2}\vert}^2 + ... + {\vert{x_n - y_n}\vert}^2 }$

In [ ]:
import numpy as np
predictions = []
for i, each in zip(x_test.index, x_test.values):  # x_test의 모든 데이터 샘플(30개)에 대한 예측값을 구하기 위해 테스트 데이터 순회
  #print(i, each)       
  distance_list = [] #  각 샘플마다 측정한 거리를 저장할 자료 구조
  for j, each_train in zip(x_train.index, x_train.values): #  현재 테스트 중인 특정 샘플에서 120개 다른 샘플에 대한 거리 측정 후 가까운 순서대로 정렬
    #print(j, each_train)
    each_point = [(xi, xj) for xi, xj in zip(each, each_train)]
    #print(each_point)
    each_distance = np.sqrt(sum([(xi - xj)*(xi - xj) for xi, xj in zip(each, each_train)]))
    distance_list.append((j, each_distance))

  # 현재 샘플의 위치에서 거리가 짧은 순서로 정렬
  # k=4 (방법1과 동일한 기준)
  # 해당 샘플과 가장 가까운 샘플 번호의 레이블 정보를 수집
  distance_list= sorted(distance_list, key=lambda x: x[1])
  index_list = [ i[0] for i in distance_list[:4]]
  
  # KNN 알고리즘과 같이 다수결로 정함 (결과가 동률인 경우 앞의 값을 선택함)
  # 최빈값을 0번 인덱스로 출력하는 pandas.series.mode() 메소드
  y_pred = df.label[index_list].mode().values[0] 
    
  predictions.append(y_pred)

print(predictions)
print(list(y_test))

[1, 0, 2, 1, 1, 0, 1, 2, 1, 1, 2, 0, 0, 0, 0, 1, 2, 1, 1, 2, 0, 2, 0, 2, 2, 2, 2, 2, 0, 0]
[1, 0, 2, 1, 1, 0, 1, 2, 1, 1, 2, 0, 0, 0, 0, 1, 2, 1, 1, 2, 0, 2, 0, 2, 2, 2, 2, 2, 0, 0]


In [ ]:
distance_list

In [ ]:
# 검증용 데이터의 샘플 인덱스 번호 확인
x_test.index, y_test.index

(Int64Index([ 73,  18, 118,  78,  76,  31,  64, 141,  68,  82, 110,  12,  36,
               9,  19,  56, 104,  69,  55, 132,  29, 127,  26, 128, 131, 145,
             108, 143,  45,  30],
            dtype='int64'),
 Int64Index([ 73,  18, 118,  78,  76,  31,  64, 141,  68,  82, 110,  12,  36,
               9,  19,  56, 104,  69,  55, 132,  29, 127,  26, 128, 131, 145,
             108, 143,  45,  30],
            dtype='int64'))

In [ ]:
# 실제값과 예측값을 비교하기 위한 데이터프레임 
df_test = pd.concat([x_test, y_test], axis=1)
df_test['y_pred_euclid_k4'] = predictions
df_test

,sl,sw,pl,pw,label,y_pred_euclid_k4
73,6.1,2.8,4.7,1.2,1,1
18,5.7,3.8,1.7,0.3,0,0
118,7.7,2.6,6.9,2.3,2,2
78,6.0,2.9,4.5,1.5,1,1
76,6.8,2.8,4.8,1.4,1,1
31,5.4,3.4,1.5,0.4,0,0
64,5.6,2.9,3.6,1.3,1,1
141,6.9,3.1,5.1,2.3,2,2
68,6.2,2.2,4.5,1.5,1,1
82,5.8,2.7,3.9,1.2,1,1


In [ ]:
# 정확도: 일치하는 값의 갯수 / 전체 개수
accuracy = ((predictions == y_test.values).sum()) / len(predictions)
accuracy

1.0

## 추가 검토 제안
- 1) 거리 기반 분류를 하는 경우 feature의 단위에 따라 모델 예측 성능이 달라질 수 있으므로 데이터 준비 단계에서 feature scaling을 수행한 뒤 KNNclassifier 모델 구현
- 2) 4차원 특성 데이터를 평면에 시각화하는 것이 어려우므로 2차원으로 낮추는 PCA 사전 전처리 후 KNNclassifier 모델 구현 

